In [26]:
import pandas as pd
from googleapiclient.discovery import build

In [27]:
# Set up the API key and YouTube API

API_KEY = open('youtube_api_key.txt', 'r').read()
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [28]:
def search_videos(query, max_results=200):
    """Search for videos on a specific topic."""
    search_response = youtube.search().list(
        q=query,
        type='video',
        part='id',
        maxResults=max_results
    ).execute()
    video_ids = [item['id']['videoId'] for item in search_response['items']]
    return video_ids

In [29]:
def get_comments(video_id):
    """Get comments for a given video ID."""
    comments = []
    # Retrieve the comments
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        textFormat='plainText',
        maxResults =500,
        order = 'relevance' #,
        #searchTerms = 'Team Fortress 2'
    ).execute()

    if response:
        for item in response['items']:
            text = item['snippet']['topLevelComment']['snippet']['textDisplay']
            channel_id = item['snippet']['topLevelComment']['snippet']['channelId']
            like_count = item['snippet']['topLevelComment']['snippet']['likeCount']
            channel_id_text = item['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
            comment = {
                'video_id': video_id,
                'text': text,
                'channel_id': channel_id,
                'like_count': like_count,
                'channel_id_text': channel_id_text
            }
            comments.append(comment)

    return comments

In [30]:
topic = "Team Fortress 2"  # Replace with the topic you want to search for
print(f"Searching for videos about: {topic}")

# Get the top 200 video IDs
video_ids = search_videos(topic, max_results=200)

all_comments = []

# Iterate through the video IDs to get comments
for video_id in video_ids:
    print(f"Fetching comments for video ID: {video_id}")
    comments = get_comments(video_id)
    all_comments.extend(comments)  # Add the comments to the list

# Print the comments (or save them to a file)
for comment in all_comments:
    print(f"- {comment}")

Searching for videos about: Team Fortress 2
Fetching comments for video ID: 7fLwk84gNPM
Fetching comments for video ID: 36lSzUMBJnc
Fetching comments for video ID: GLlLQ3LmZWU
Fetching comments for video ID: EnFHmRxj4hA
Fetching comments for video ID: IAytdaXR5gw
Fetching comments for video ID: YzcBMfRSpEE
Fetching comments for video ID: hB0uIFcLRbc
Fetching comments for video ID: N7ZafWA2jd8
Fetching comments for video ID: QxVFPlsowtI
Fetching comments for video ID: KFXHtubsCRg
Fetching comments for video ID: BHYKNVAsXpk
Fetching comments for video ID: 2d5kIJDHVZU
Fetching comments for video ID: bMWQMT9096g
Fetching comments for video ID: lxPKzk4j93k
Fetching comments for video ID: n90PfCri8ds
Fetching comments for video ID: WUhOnX8qt3I
Fetching comments for video ID: 3riShkiZd_s
Fetching comments for video ID: ZvfPhpDRp_w
Fetching comments for video ID: AzjmHaYK9go
Fetching comments for video ID: 1MezglHkjYg
Fetching comments for video ID: Zwj2GTLCWs4
Fetching comments for video ID: 

In [31]:
# Convert all_comments to a df with columns as keys in the dictionary and values as each row
df = pd.DataFrame(all_comments)

In [32]:
df.head()

,video_id,text,channel_id,like_count,channel_id_text
0,7fLwk84gNPM,*Have a great day! :)*,UCBFl4uTDFRlIliFk3jqSLQg,15,UCBFl4uTDFRlIliFk3jqSLQg
1,7fLwk84gNPM,"I love your gameplays, the way you use the voi...",UCBFl4uTDFRlIliFk3jqSLQg,11,UCQCOAbxmVH-znIErvHK7KYA
2,7fLwk84gNPM,21:50 it wat in that moment he knew...,UCBFl4uTDFRlIliFk3jqSLQg,6,UCkmUnkgNw5Jcy2aGIgLZRdg
3,7fLwk84gNPM,I was in a game with this spy yesterday (enemy...,UCBFl4uTDFRlIliFk3jqSLQg,0,UCrI0jKZIE0mn_pOMvw7_Uaw
4,7fLwk84gNPM,Let’s gooo YER gameplay!! My fav! :D,UCBFl4uTDFRlIliFk3jqSLQg,3,UC0y5Rw8xFO_2f9Gfatn4Mrw


In [33]:
df.to_csv('youtube_comments.csv')